In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from core import CompanyProduct

target = CompanyProduct.same("98point6")

In [3]:
from glassdoor.search import find_review_url

url = find_review_url(target, debug=True)

['https://www.glassdoor.com/Reviews/98point6-Reviews-E1181484.htm', 'https://www.glassdoor.com/Reviews/98point6-Seattle-Reviews-EI_IE1181484.0,8_IL.9,16_IM781.htm', 'https://www.glassdoor.com/Reviews/98point6-Reviews-E1181484.htm?filter.jobTitleExact=Software+Engineer(Internship)', 'https://www.glassdoor.com/Reviews/98point6-layoff-Reviews-EI_IE1181484.0,8_KH9,15.htm', 'https://www.glassdoor.com/Reviews/98point6-Physician-Reviews-EI_IE1181484.0,8_KO9,18.htm', 'https://www.glassdoor.com/Reviews/98point6-98point6-Reviews-EI_IE1181484.0,8_KH9,17.htm']


In [4]:
from glassdoor.scraper import scrape_reviews, GlassdoorReview

glassdoor_results = await scrape_reviews(url, max_pages=3)
reviews = GlassdoorReview.parse_reviews(glassdoor_results)


2024-07-30 08:42:16.902 | INFO     | glassdoor.scraper:scrape_reviews:110 - scraping reviews from https://www.glassdoor.com/Reviews/98point6-Reviews-E1181484.htm
2024-07-30 08:42:18.171 | INFO     | glassdoor.scraper:scrape_reviews:118 - scraped first page of reviews of https://www.glassdoor.com/Reviews/98point6-Reviews-E1181484.htm, scraping remaining 2 pages
INFO:backoff:Backing off scrape(...) for 0.4s (requests.exceptions.ReadTimeout: HTTPSConnectionPool(host='api.scrapfly.io', port=443): Read timed out. (read timeout=160))
2024-07-30 08:45:10.531 | INFO     | glassdoor.scraper:scrape_reviews:128 - scraped 30 reviews from https://www.glassdoor.com/Reviews/98point6-Reviews-E1181484.htm in 3 pages


In [5]:
from glassdoor.summarizer import summarize

summary = summarize(target, reviews)
print(summary.content)

The prompt context has 13,916 characters in 30 reviews
# Summary of 98point6 Glassdoor Reviews

## Key Aspects

### Leadership
**Pros:** Some reviews highlight strong leadership at the team level, with managers who are responsive and supportive.  
**Cons:** Many reviews criticize upper management for being disconnected, ineffective, and lacking emotional intelligence. There is a noted "boys' club" mentality and a failure to listen to employee feedback.  
**Quotations:** "The management isn’t great- their inexperience in such roles definitely show in really cruddy ways." (Licensed Mental Health Professional on 2023-01-31)

### Compensation and Benefits
**Pros:** Employees appreciate the generous PTO and flexible work arrangements.  
**Cons:** Compensation is frequently described as low, especially for the workload expected. Many employees have left for better pay elsewhere.  
**Quotations:** "The biggest two factors are terrible managers and extremely weak pay." (Anonymous on 2022-03-13